## Overview
Here we use ASPECT to numerically reproduce the results of a linear stability analysis for the onset of convection in a fluid layer heated from below. This exercise was assigned to students in a geodynamics class at Portland State University as a first step towards setting up a nominally Earth-like mantle convection model. Hence, representative length scales and transport properties for Earth are used.

The linear stability analysis appears in Turcotte and Schubert (2014) section 6.19. To use this code, you must compile aspect and give the path to the executable below as $\texttt{aspect_bin}$. The critical Rayleigh number for the onset of convection depends only on the dimensionless wavelength of the perturbation, which is assumed to be equal to width of the domain. The domain has height $b$ and width $\lambda$ and the perturbation has the functional form:

$
T'(x,y) = T_0'\cos\left(\frac{2\pi x}{\lambda}\right)\sin\left(\frac{\pi y}{b} \right)
$

Note that because we place the bottom boundary of the domain at $y=0$ and the top at $y=b$, the perturbation vanishes at the top and bottom boundaries.

The analytic solution for the critical Rayleigh number, $Ra_{cr}$ is given in T\&S equation 6.319:

$
Ra_{cr}=\frac{\left(\pi^2+\frac{4\pi^2 b^2}{\lambda^2}\right)^3}{\frac{4\pi^2 b^2}{\lambda^2}}
$

The linear stability analysis also makes a prediction for the dimensionless growth rate of the instability $\alpha'$. The maximum vertical velocity is given by:

$
v_{y,max} = \frac{2\pi}{\lambda}\phi_0' e^{\alpha' t}
$,

where

$
\phi_0' = -\frac{2\pi}{\lambda}\frac{\rho_0 g \alpha T_0'}{\mu}\left(\frac{4\pi^2}{\lambda^2}+\frac{\pi^2}{b^2} \right)^{-2}
$,

and

$
\alpha'=\frac{\kappa}{b^2}\left[\frac{\rho_0 g \alpha b^3 \Delta T}{\mu \kappa}\left(\frac{\frac{4\pi^2 b^2}{\lambda^2}}{\left(\frac{4\pi^2 b^2}{\lambda^2}+\pi^2\right)^2}\right) -\left(\pi^2+\frac{4\pi^2b^2}{\lambda^2}\right)\right]
$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from subprocess import run

base_input = "convection-box-base.prm"     # The 'base' input file that gets modified
input_file = "input.prm"                   # The temporary input file that is used for each run
aspect_bin = "../../build/aspect"          # Path to aspect executable
output_dir = "output"                      # Output directory

In [ ]:
# Create a dictionary to replace keys in base input file with appropriate values
parameters = dict([])
parameters['PMAG'] = 1                # amplitude of temperature perturbation
parameters['HEIGHT'] = 3.0e6          # layer thickness
parameters['IREF'] = 1              # initial global refinement
parameters['DELTA_T'] = 2500.0        # temperature difference between bottom, top boundaries
parameters['GRAVITY'] = 10.0          # vertical gravitational acceleration
parameters['KTHERMAL'] = 4.0          # thermal conductivity
parameters['ALPHA'] = 3e-5            # thermal expansivity
parameters['DENSITY'] = 4000.0        # reference density
parameters['SPECIFIC_HEAT'] = 1250.0  # specific heat capacity
parameters['NY'] = 16;                # number of x-repetitions

def generate_input_file(base_file_name,output_file_name,dictionary):
    """Read the 'base' input file from base_file_name, replace strings 
    using dictionary, and write new output file to output_file_name"""
    fh = open(base_file_name,'r')
    run(['rm','-f',output_file_name])
    ofh = open(output_file_name,'w')
    for line in fh:        
        for key in dictionary:
            if key in line:                
                line = line.replace(key,str(dictionary[key]))
        ofh.write(line)
    fh.close()
    ofh.close()
    
def parse_output(output_dir):
    """Read the statistics (stats_file) file from the output directory"""
    tmp = np.loadtxt(output_dir + '/statistics',usecols=(1, 11))
    vy = np.loadtxt(output_dir + '/point_values.txt',usecols=(4,))
    t=tmp[:,0]
    vmax = tmp[:,1]
    return t,vmax,vy
    
def vy_exact(t,p):
    """Evaluate exact expression for maximum upward velocity"""
    lam = p['WIDTH']
    b = p['HEIGHT']
    g = p['GRAVITY']
    T0 = p['PMAG']
    alpha = p['ALPHA']
    eta = p['VISCOSITY']
    rho = p['DENSITY']
    dT = p['DELTA_T']
    kappa = p['KTHERMAL']/rho/p['SPECIFIC_HEAT']
    alpha_prime = kappa/b**2*((rho*g*alpha*b**3*dT/eta/kappa)*(4.0*np.pi**2*b**2/lam**2/(4.0*np.pi**2*b**2/lam**2+np.pi**2)**2)-(np.pi**2+4.0*np.pi**2*b**2/lam**2))
    phi0 = -2.0*np.pi/lam*(rho*g*alpha*T0/eta)*(4.*np.pi**2/lam**2 + np.pi**2/b**2)**(-2.0)
    vymax = 2.0*np.pi/lam*phi0 * np.exp(alpha_prime*t)
    return vymax
    
def run_aspect(rayleigh,p=parameters):
    """Perofm the following tasks. 
    1. Choose viscosity to satisfy rayleigh number rayleigh and parameters in p, which
    defaults to the parameters dictionary defined in the current namespace. 
    2. Generate an input file. 
    3. Run aspect
    4. Parse the results, return the rate at which velocities increase between the first and second timesteps."""
    def eta(ra):        
        rho = p['DENSITY']
        alpha = p['ALPHA']
        g = p['GRAVITY']
        dT = p['DELTA_T']
        h = p['HEIGHT']     
        kappa = p['KTHERMAL']/p['DENSITY']/p['SPECIFIC_HEAT']    
        return rho*g*alpha*dT*(h**3)/ra/kappa                
    
    p['VISCOSITY'] = eta(rayleigh);
    # Change the number of x-repeitions to keep elements close to equant
    p['NX'] = int(p['WIDTH']/p['HEIGHT']*p['NY'])
    p['EVALUATION_POINTS'] =  str(0.0) + ',' + str(p['HEIGHT']/2.0)
    generate_input_file(base_input,input_file,p)
    run(['rm','-rf',output_dir])
    run([aspect_bin,input_file])
    t, vmax, vy = parse_output(output_dir)
    return (vmax[1]-vmax[0]), t, vy



Define a function to perform bisection: 

In [ ]:
def bisection(function, low, high, atol=np.Inf, rtol=0.01):
    """Performs bisection to find 0.0=function(x) using function and initial bounds (low, high) 
    to within absolute value tolerance atol OR relative tolerance rtol"""
    assert function(low)[0]<0 
    assert function(high)[0]>0
    x_try = [];
    y_try = [];
    
    while abs(low-high)>atol or abs(low-high)/abs(low) > rtol:
        x = (low + high)/2.0
        x_try.append(x)
        y = function(x)[0]       
        y_try.append(y)
        if y > 0:
            high=x;
        else:
            low=x;        
    return x, x_try, y_try

Define the range of aspect ratios for which $Ra_{cr}$ is calculated. Iterate over them and perform bisection to determine $Ra_{cr}$ for each.

In [ ]:
aspect_min = np.pi/4.
aspect_max = 12.*np.pi
naspect = 10;
ra_rtol = 1e-5;

def racr_lsa( b_over_lam ):
    """Evaluates the critical Rayleigh number for b_over_lam, which is the ratio of box height 
    to box width. Implements Turcotte and Schubert (3ed) equation 6.319"""
    pi = np.pi
    racr = (pi**2 + 4*pi**2*b_over_lam**2)**3/(4*pi**2*b_over_lam**2)
    return racr

aspect_ratio = np.linspace(1.0/aspect_max,1.0/aspect_min,naspect)
aspect_ratio = 1.0/aspect_ratio
lams=[]
ra_save = []
val_save = []
racr_save = []
xplot = [];
for a in aspect_ratio:
    # calculate dimensional width
    parameters['WIDTH'] = a*parameters['HEIGHT']
    racr_analytic =racr_lsa( 1.0/a )
    racr_low  = racr_analytic/3.0
    racr_high = racr_analytic*3.0
    racr, ratry, vals = bisection(run_aspect,racr_low,racr_high,rtol=ra_rtol)
    xplot.append( 2.0*np.pi*parameters['HEIGHT']/parameters['WIDTH'] )
    ra_save.append(ratry)
    racr_save.append(racr)
    val_save.append(vals)


In [ ]:
print(parameters['WIDTH']/parameters['HEIGHT'])
print(run_aspect(racr_high))
print(run_aspect(racr_low))

In [ ]:
def symbol(value):
    """Returns a string specifying figure glyph and color corresponding to value"""
    if value>0:
        return 'ro'
    else:
        return 'k.'

plt.figure()
for j in range(len(ra_save)):
    # Plot the results
    ratry = ra_save[j]
    dvdt = val_save[j]
    
    for i in range(len(ratry)):
        shape, mycolor = symbol(ratry[i])
        plt.plot(xplot[j],ratry[i],symbol(dvdt[i]))

# Plot the analytic solution.
xp = np.linspace(xplot[0],xplot[-1],1000)
plt.plot(xp,[racr_lsa(x/(2*np.pi)) for x in xp],'g:')
plt.text(2,3000,'Unstable')
plt.text(5.5,1000,'Stable')
plt.xlabel('$2\pi b/\lambda$')
plt.ylabel('$Ra_{cr}$')
plt.ylim([0, 4500])

plt.savefig('racr.png', bbox_inches='tight')
plt.show()

In [ ]:
# Make a plot showing the relative error between numerical and analytic solutions
# Recall that we should expect the level agreement to be limited by the relative error
# tolerance defined during the bisection procedure.
plt.figure()
racr = np.array([racr_lsa(x) for x in xplot])
e_norm = [np.abs(racr_save[i]-racr_lsa(xplot[i]/(2.*np.pi)))/np.abs(racr_lsa(xplot[i]/(2.*np.pi))) for i in range(len(xplot))]
plt.plot(xplot,e_norm,'k.')
plt.plot([0,8],[ra_rtol,ra_rtol],'r--')
plt.xlabel('$2\pi b/\lambda$')
plt.ylabel('relative error')
plt.yscale('log')
plt.savefig('racr_error.png', bbox_inches='tight')
plt.show()

In [ ]:
# As an additional example, compare the analytic and numerically determined maximal velocities
parameters['WIDTH'] = 2.0*np.sqrt(2.0)*parameters['HEIGHT'];
ra_values = np.linspace(250,1000,10)
dvdt=[]
vy0=[];  #Numerically determined vy at time 0
vy1=[];  #Numerically determined vy at time 1
vye0=[]; #Exact vy at time 0
vye1=[]; #Exact vy at time 1
for ra in ra_values:    
    tmp1,tmp2,tmp3 = run_aspect(ra,parameters)
    t0=tmp2[0]
    t1=tmp2[1]
    vye0.append( vy_exact(t0,parameters) )
    vye1.append( vy_exact(t1,parameters) )
    dvdt.append(tmp1)
    vy0.append(tmp3[0])
    vy1.append(tmp3[1])
    
err0 = [abs(vy0[i]-vye0[i])/abs(vye0[i]) for i in range(len(vy0))]
err1 = [abs(vy1[i]-vye1[i])/abs(vye1[i]) for i in range(len(vy1))]
plt.figure()
plt.plot(ra_values,err0,'rx',label='Time 0')
plt.plot(ra_values,err1,'kx',label='Time 1')
plt.yscale('log')
plt.legend()
plt.ylabel('Relative Error')
plt.xlabel('Rayleigh Number')
plt.show()